SAS 
Endpoint=sb://labb2-eh-namespace.servicebus.windows.net/;SharedAccessKeyName=labb2-eh-key;SharedAccessKey=SdrGwZXoUxwg6fP812yyboUv9SH//94Zf+AEhCYAUB4=;EntityPath=labb2-event-hub

In [0]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType



In [0]:
connectionString = "Endpoint=sb://labb2-eh-namespace.servicebus.windows.net/;SharedAccessKeyName=labb2-eh-key;SharedAccessKey=SdrGwZXoUxwg6fP812yyboUv9SH//94Zf+AEhCYAUB4=;EntityPath=labb2-event-hub"

ehConf = {
  'eventhubs.connectionString' : connectionString
}

# Read stream from Event Hub
df = (
  spark.readStream
    .format("eventhubs")
    .options(**ehConf)
    .load()
)

# Event Hub body is binary, so decode to string
df = df.withColumn("body", col("body").cast(StringType()))

# Show the schema
df.printSchema()


In [0]:
# Extract JSON from Body
# Define your schema (adjust to your JSON structure)
json_schema = StructType([
    StructField("id", StringType()),
    StructField("name", StringType()),
    StructField("value", StringType())
])

parsed_df = df.withColumn("jsonData", from_json(col("body"), json_schema))

In [0]:
# Select fields
final_df = parsed_df.select("jsonData.*", "enqueuedTime")


In [0]:
# Debugging
query = (
  final_df.writeStream
    .outputMode("append")
    .format("console")
    .option("truncate", False)
    .start()
)


In [0]:
# Sink to Storage / Delta Lake
query = (
  final_df.writeStream
    .outputMode("append")
    .format("delta")
    .option("checkpointLocation", "/mnt/checkpoints/myeventhub")
    .table("eventhub_raw")
)
